In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras as keras
from keras.models import Sequential
from keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from os import listdir
import math

Differenze tra still, walking e running

In [ ]:
# relevant features
labels = ['xa', 'ya', 'za', 'xg', 'yg', 'zg']

still = pd.read_csv('datasets/still_19_02_2023_17_28_18.csv')
still = still[labels]

walking = pd.read_csv('datasets/walking_19_02_2023_17_10_11.csv')
walking = walking[labels]

running = pd.read_csv('datasets/running_23_02_2023_10_48_27.csv')
running = running[labels]

fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9), (ax10, ax11, ax12), (ax13, ax14, ax15), (ax16, ax17, ax18)) = plt.subplots(6,3)

fig.set_figheight(30)
fig.set_figwidth(20)

ax1.plot(np.arange(0, still.shape[0]), still['xa'])
ax1.set_title('Accelerometer X-axis (Still)')

ax2.plot(np.arange(0, walking.shape[0]), walking['xa'])
ax2.set_title('Accelerometer X-axis (Walking)')

ax3.plot(np.arange(0, running.shape[0]), running['xa'])
ax3.set_title('Accelerometer X-axis (Running)')


ax4.plot(np.arange(0, still.shape[0]), still['ya'])
ax4.set_title('Accelerometer Y-axis (Still)')

ax5.plot(np.arange(0, walking.shape[0]), walking['ya'])
ax5.set_title('Accelerometer Y-axis (Walking)')

ax6.plot(np.arange(0, running.shape[0]), running['ya'])
ax6.set_title('Accelerometer Y-axis (Running)')


ax7.plot(np.arange(0, still.shape[0]), still['za'])
ax7.set_title('Accelerometer Z-axis (Still)')

ax8.plot(np.arange(0, walking.shape[0]), walking['za'])
ax8.set_title('Accelerometer Z-axis (Walking)')

ax9.plot(np.arange(0, running.shape[0]), running['za'])
ax9.set_title('Accelerometer Z-axis (Running)')

ax10.plot(np.arange(0, still.shape[0]), still['xg'])
ax10.set_title('Gyroscope X-axis (Still)')

ax11.plot(np.arange(0, walking.shape[0]), walking['xg'])
ax11.set_title('Gyroscope X-axis (Walking)')

ax12.plot(np.arange(0, running.shape[0]), running['xg'])
ax12.set_title('Gyroscope X-axis (Running)')


ax13.plot(np.arange(0, still.shape[0]), still['yg'])
ax13.set_title('Gyroscope Y-axis (Still)')

ax14.plot(np.arange(0, walking.shape[0]), walking['yg'])
ax14.set_title('Gyroscope Y-axis (Walking)')

ax15.plot(np.arange(0, running.shape[0]), running['yg'])
ax15.set_title('Gyroscope Y-axis (Running)')

ax16.plot(np.arange(0, still.shape[0]), still['zg'])
ax16.set_title('Gyroscope Z-axis (Still)')

ax17.plot(np.arange(0, walking.shape[0]), walking['zg'])
ax17.set_title('Gyroscope Z-axis (Walking)')

ax18.plot(np.arange(0, running.shape[0]), running['zg'])
ax18.set_title('Gyroscope Z-axis (Running)')



Lettura dei dataset dalla cartella datasets

In [ ]:
# list of files inside datasets folder
files = listdir('datasets')

datasets = []
# datasets_train = []
# datasets_test = []

# for every file add a label
def divide_and_add_activity(name, df):
    labels = ['xa', 'ya', 'za', 'xg', 'yg', 'zg']
    # ridivido per 1000 (i dati erano stati moltiplicati per 1000 prima di essere inviati dalla board in modo da spedire
    # numeri interi anzichè dei numeri in virgola mobile)
    # df[labels] = df[labels] / 1000

    # aggiungo la label dell'attvità
    if 'still' in name:
        df['activity'] = 'still'
    elif 'walking' in name:
        df['activity'] = 'walking'
    else:
        df['activity'] = 'running'
    return df

# def split_data(df):
#     num_rows = df.shape[0]
#     num_test = math.floor(num_rows / 3)
#     num_train = num_rows - num_test

#     df_train = df.iloc[:num_train, :]
#     df_test = df.iloc[num_train:, :]

#     datasets_train.append(df_train)
#     datasets_test.append(df_test)

#     print(df_train.shape)
#     print(df_test.shape)

# add activity to the labels we are interested
labels = ['xa', 'ya', 'za', 'xg', 'yg', 'zg', 'activity']

for file in files:
    df = divide_and_add_activity(file, pd.read_csv(f'datasets/{file}'))
    df = df[labels]
    datasets.append(df)
    print("Dimensione dataset {} ".format(file), df.shape)

# for df in datasets:
#     split_data(df)

# df_train = pd.concat(datasets_train, ignore_index=True)
# df_test = pd.concat(datasets_test, ignore_index=True)

df_data = pd.concat(datasets)

# print("Dimensione merged dataset train ", df_train.shape)
# print("Dimensione merged dataset test ", df_test.shape)
print("Dimensione merged dataset", df_data.shape)
print(df_data)



Normalizzazione dei dati

In [ ]:
normalizer = MinMaxScaler()
num_columns = df_data.shape[1]

normalizer.fit(df_data.iloc[:, 0 : num_columns-1])

def normalize_df(df):
    
    norm_df = df.copy()
    norm_column = normalizer.transform(df.iloc[:, 0 : num_columns-1])
    norm_df.iloc[:, 0 : num_columns-1] = norm_column

    return norm_df

# norm_train_df = normalize_df(df_train)
# norm_test_df = normalize_df(df_test)




def arduino_normalizer(df):
    norm_df = df.copy()
    labels = ['xa', 'ya', 'za', 'xg', 'yg', 'zg']
    norm_df[labels] = norm_df[labels] / 1000.0
    norm_df['xa'] = (norm_df['xa'] + 4.0) / 8.0
    norm_df['ya'] = (norm_df['ya'] + 4.0) / 8.0
    norm_df['za'] = (norm_df['za'] + 4.0) / 8.0
    norm_df['xg'] = (norm_df['xg'] + 2000.0) / 4000.0
    norm_df['yg'] = (norm_df['yg'] + 2000.0) / 4000.0
    norm_df['zg'] = (norm_df['zg'] + 2000.0) / 4000.0
    return norm_df


norm_df = normalize_df(df_data)

fig, ((ax1, ax2),( ax3, ax4), (ax5, ax6)) = plt.subplots(3,2)
ax1.plot(np.arange(0, len(df_data.xa)), df_data.xa)
ax3.plot(np.arange(0, len(df_data.ya)), df_data.ya)
ax5.plot(np.arange(0, len(df_data.za)), df_data.za)

ax2.plot(np.arange(0, len(norm_df.xa)), norm_df.xa)
ax4.plot(np.arange(0, len(norm_df.ya)), norm_df.ya)
ax6.plot(np.arange(0, len(norm_df.za)), norm_df.za)



norm_df

Resize datasets

In [ ]:
def reshape_data(df, time_steps, step):
    Xs = []
    ys = []

    num_cols = df.shape[1]

    X = df.iloc[:, :num_cols - 1]
    y = df.iloc[:, num_cols - 1:]

    print(X.shape)

    for i in range(0, len(X) - time_steps, step):
        values = X.iloc[i : (i+time_steps)].values
        labels = y.iloc[i : i + time_steps]
        Xs.append(values)
        ys.append(labels.mode()['activity'][0])
    
    return np.array(Xs), np.array(ys).reshape(-1, 1)


# X_train , y_train = reshape_data(norm_train_df, 20, 20)
# X_test , y_test = reshape_data(norm_test_df, 20, 20)

X_data, y_data = reshape_data(norm_df, 20, 20)
print(X_data.shape)


Split test and train data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33, random_state=42)

print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

OneHotEncoder per la y

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(y_train)

def encode_y(y):
    encoded_y = encoder.transform(y)
    return encoded_y

encoded_y_train = encode_y(y_train)
encoded_y_test = encode_y(y_test)


print(encoder.categories_)
# print(y_train)
print(pd.DataFrame(encoded_y_train, columns=encoder.categories_))

In [ ]:
# stampa delle dimensioni

print(X_train.shape)
print(X_test.shape)

print(encoded_y_train.shape)
print(encoded_y_test.shape)
encoded_y_train

# Modello

In [ ]:
model = Sequential()
model.add(Dense(16, input_shape=[X_train.shape[1], X_train.shape[2]], activation='relu'))
# model.add(LSTM(units=8, input_shape=[X_train.shape[1], X_train.shape[2]]))
# model.add(Dropout(rate=0.3))
# model.add(BatchNormalization())
model.add(Dense(32, activation = 'relu'))
# model.add(Dropout(rate=0.3))
# model.add(BatchNormalization())
model.add(Dense(32, activation = 'relu'))
# model.add(Dropout(rate=0.3))
# model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(encoded_y_train.shape[1], activation = 'softmax'))
print(model.summary())


Modello - ottimizzatore e funzione di perdita

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

Early stopping per evitare overfitting

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train, encoded_y_train,
    validation_data=(X_test, encoded_y_test),
    # validation_split=0.1,
    batch_size=3,
    epochs=100,
    callbacks=[early_stopping]
)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

Matrice per l'affidabilità della predizione sui dati di validazione

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
def show_confusion_matrix(validations, predictions):

    matrix = confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=['running', 'still', 'walking'],
                yticklabels=['running', 'still', 'walking'],
                annot=True,
                fmt='d')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

y_pred_test = model.predict(X_test)

# indice della massima probabilità
max_y_pred_test = np.argmax(y_pred_test, axis=1)

max_y_test = np.argmax(encoded_y_test, axis=1)

show_confusion_matrix(max_y_test, max_y_pred_test)

print(classification_report(max_y_test, max_y_pred_test))

In [ ]:
model.evaluate(X_test, encoded_y_test)

piccolo test con alcuni dati in input

In [ ]:
data_still = pd.read_csv('datasets/running_23_02_2023_10_48_27.csv')[labels[0:-1]]
data_norm = normalize_df(data_still)

data_input = np.array([data_norm.iloc[7:27,:], data_norm.iloc[30:50,:]])
print(data_input.shape)
print(data_input)

res = model.predict(data_input)
print(res)
print(np.argmax(res[0]))

In [ ]:
data_still = pd.read_csv('datasets/running_23_02_2023_10_48_27.csv')[labels[0:-1]]

data_norm = normalize_df(data_still)

data_input = []

count_still = 0
count_walking = 0
count_running = 0

# resize data in signals of 20 samples
for i in range(0, data_norm.shape[0]-20, 20):
    data_input.append(data_norm.iloc[i:(i+20), :])

input = np.array(data_input)
print(input.shape)

# classify the input
res = model.predict(input)
print(res)

# get the index of the biggest probability (0 = still, 1 = walking)
res = np.argmax(res, axis=1)
print(res)

for value in res:
    if value == 0:
        count_running += 1
    elif value == 1:
        count_still +=1
    else:
        count_walking += 1

print('Still: ', count_still)
print('Walking: ', count_walking)
print('Running: ', count_running)




In [ ]:
model.save('activity_classifier_model')

# convert model to tensorflow lite
converter = tf.lite.TFLiteConverter.from_saved_model('activity_classifier_model')
tflite_model = converter.convert()

# Save the model to disk
open("./activity_classifier.tflite", "wb").write(tflite_model)


import os
basic_model_size = os.path.getsize("activity_classifier.tflite")
print("Model is %d bytes" % basic_model_size)

# da eseguire in un ambiente linux o colab
# !echo "const unsigned char model[] = {" > /content/model.h
# !cat gesture_model.tflite | xxd -i      >> /content/model.h
# !echo "};"                              >> /content/model.h

# import os
# model_h_size = os.path.getsize("model.h")
# print(f"Header file, model.h, is {model_h_size:,} bytes.")
# print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")